### Import `libraries`

In [1]:
from time import sleep
import time
from json import dumps
from kafka import KafkaProducer 
from kafka.errors import KafkaError
import pandas as pd

### Using `Smoke Detection Data`

In [2]:
def CreateDataFrameForSmokeDetectionData():
    SmokeDetectionData = r''
    dfSDD = pd.read_csv(SmokeDetectionData, index_col=0)
    return dfSDD

dfSDD = CreateDataFrameForSmokeDetectionData()
dfSDD

,UTC,Temperature[C],Humidity[%],TVOC[ppb],eCO2[ppm],Raw H2,Raw Ethanol,Pressure[hPa],PM1.0,PM2.5,NC0.5,NC1.0,NC2.5,CNT,Fire Alarm
0,1654733331,20.000,57.36,0,400,12306,18520,939.735,0.00,0.00,0.00,0.000,0.000,0,0
1,1654733332,20.015,56.67,0,400,12345,18651,939.744,0.00,0.00,0.00,0.000,0.000,1,0
2,1654733333,20.029,55.96,0,400,12374,18764,939.738,0.00,0.00,0.00,0.000,0.000,2,0
3,1654733334,20.044,55.28,0,400,12390,18849,939.736,0.00,0.00,0.00,0.000,0.000,3,0
4,1654733335,20.059,54.69,0,400,12403,18921,939.744,0.00,0.00,0.00,0.000,0.000,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62625,1655130047,18.438,15.79,625,400,13723,20569,936.670,0.63,0.65,4.32,0.673,0.015,5739,0
62626,1655130048,18.653,15.87,612,400,13731,20588,936.678,0.61,0.63,4.18,0.652,0.015,5740,0
62627,1655130049,18.867,15.84,627,400,13725,20582,936.687,0.57,0.60,3.95,0.617,0.014,5741,0
62628,1655130050,19.083,16.04,638,400,13712,20566,936.680,0.57,0.59,3.92,0.611,0.014,5742,0


### Convert `DataFrame` to a JSON format

In [3]:
def CreateJSONformatFromSDD():
    return dfSDD.sample(1).to_dict(orient='records')[0]

JSONFormat = CreateJSONformatFromSDD()    
JSONFormat

{'UTC': 1654785440,
 'Temperature[C]': 24.9,
 'Humidity[%]': 50.59,
 'TVOC[ppb]': 1199,
 'eCO2[ppm]': 400,
 'Raw H2': 13023,
 'Raw Ethanol': 19425,
 'Pressure[hPa]': 938.774,
 'PM1.0': 2.67,
 'PM2.5': 2.78,
 'NC0.5': 18.39,
 'NC1.0': 2.867,
 'NC2.5': 0.065,
 'CNT': 24097,
 'Fire Alarm': 1}

### Configuring a `kafka client` that published records to the Kafka Cluster


In [4]:
def CreateKafkaProducer():
    localhost = 'localhost:9092' # host that the producer should contact to boostrap initial cluste metadata
    return KafkaProducer(bootstrap_servers=[localhost], value_serializer=lambda x: dumps(x).encode('utf-8')) # it publishes events to Kafka

### Publishing messages (Smoke Detection Data) to a `topic`

In [5]:
def PublishingMessageSSDToTopic():

    producer = CreateKafkaProducer()
    topic_name = 'SmokeDetection' # topic where the message will be published
    end_time = time.time() + 15 # the script will run for 15 seconds
    while True:
        if time.time() > end_time:
            break
        SDD = CreateJSONformatFromSDD()    
        producer.send(topic_name, value=SDD)

PublishingMessageSSDToTopic()

### Shutting down the producer so as to ensure that all `messages were delivered`

In [6]:
def ProducerFlush():
    producer = CreateKafkaProducer()
    return producer.flush()

ProducerFlush()